# <div align="center"><b> YOLOV11 - TEST </b></div>

<div align="right">📝 <em><small><font color='Gray'>Nota:</font></small></em></div>

<div align="right"> <em><small><font color='Gray'> La funcionalidad de visualización de jupyter notebooks en <a href="https://github.com/" target="_blank">github</a> es solamente un preview.</font></small></em> </div>

<div align="right"> <em><small><font color='Gray'> Para mejor visualización se sugiere utilizar el visualizador recomendado por la comunidad: <a href="https://nbviewer.org/" target="_blank">nbviewer</a></font></small></em> </div>

<div align="right"> <em><small><font color='Gray'> Puedes a acceder al siguiente enlace para ver este notebook en dicha página: <a href="https://nbviewer.org/ruta/de/archivo.ipynb">Ruta archivo</a></font></small></em> </div>

* * *

<style>
/* Limitar la altura de las celdas de salida en html */
.jp-OutputArea.jp-Cell-outputArea {
    max-height: 500px;
}
</style>

🛻 <em><font color='MediumSeaGreen'>  Instalaciones: </font></em> 🛻

Este notebook utiliza [Poetry](https://python-poetry.org/) para la gestión de dependencias.
Primero instala Poetry siguiendo las instrucciones de su [documentación oficial](https://python-poetry.org/docs/#installation).
Luego ejecuta el siguiente comando para instalar las dependencias necesarias y activar el entorno virtual:

- Bash:
```bash
poetry install
eval $(poetry env activate)
```

- PowerShell:
```powershell
poetry install
Invoke-Expression (poetry env activate)
```

<!-- Descargar archivos adicionales:
!gdown https://drive.google.com/drive/folders/1UBZ8PEbtmiWMGkULu7GAt3VhUpeTy9l7?usp=sharing --folder -->

In [1]:
# Chequear versión de CUDA
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2024 NVIDIA Corporation
Built on Wed_Oct_30_01:18:48_Pacific_Daylight_Time_2024
Cuda compilation tools, release 12.6, V12.6.85
Build cuda_12.6.r12.6/compiler.35059454_0


In [2]:
# Chequear más datos sobre la GPU
!nvidia-smi

Sat May 24 16:14:06 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 566.36                 Driver Version: 566.36         CUDA Version: 12.7     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                  Driver-Model | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 4080 ...  WDDM  |   00000000:01:00.0 Off |                  N/A |
|  0%   34C    P8              5W /  320W |    1267MiB /  16376MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

✋ <em><font color='DodgerBlue'>Importaciones:</font></em> ✋

In [3]:
import sys, os, json, uuid
from dotenv import load_dotenv

load_dotenv("../../.env.dev")

sys.path.append(os.path.abspath("../../"))  # Se agrega utilidades generales del modulo IA
sys.path.append(os.path.abspath("../../../modulo-apps"))  # Se agrega modulo-mini-apps

from typing import List, Dict, Any, Optional
from pathlib import Path

import ultralytics
from ultralytics import YOLO
import torch

import fiftyone as fo

import cv2 as cv

from apps_utils.logging import Logging
from apps_config.settings import Config
from apps_com_db.mongodb_client import MongoDB

import ia_utils.dataset_utils as DatasetUtils
import ia_utils.gpu_utils as GPUUtils
import ia_utils.yolo_dataset_utils as YoloDatasetUtils

from apps_com_s3.procesador_s3 import ProcesadorS3
import apps_etiquetado.procesador_anotaciones_mongodb as ProcesadorCocoDataset
import apps_etiquetado.procesador_anotaciones_coco_dataset as UtilsCocoDataset

🔧 <em><font color='tomato'>Configuraciones:</font></em> 🔧


In [4]:
# Crear instancia de Config
CONFIG = Config().config_data
LOGGER = Logging().logger
DB = MongoDB().db

download_folder = Path(CONFIG["folders"]["download_folder"])
DOWNLOAD_RAW_DATASET_FOLDER = Path(CONFIG["folders"]["raw_dataset_folder"])
DOWNLOAD_YOLO_DATASET_FOLDER = Path(CONFIG["folders"]["yolo_dataset"])
DOWNLOAD_PREDICTION_FOLDER = download_folder / "predictions"

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"  # Establece el dispositivo.

# Parámetros
BATCH_SIZE = 10  # Tamaño del batch
N_EPOCHS = 10  # Número de épocas
VERBOSE = True  # Muestra época a época la evolución
RANDOM_SEED = 42  # Semilla para la aleatoriedad

print(f"Dispositivo actual: {DEVICE}")
ultralytics.checks()  # Verifica la instalación de ultralytics

Ultralytics 8.3.140  Python-3.13.3 torch-2.7.0+cu126 CUDA:0 (NVIDIA GeForce RTX 4080 SUPER, 16376MiB)
Setup complete  (20 CPUs, 127.9 GB RAM, 637.2/1862.9 GB disk)


<div align="center">✨Datos del proyecto:✨</div>

<p></p>

<div align="center">

| Subtitulo       | YoloV11 - Detección de palmeras                                                                                                                |
| --------------- | -------------------------------------------------------------------------------------------------------------------------------------- |
| **Descrpción**  | Notebook de detección de palmeras utilizando YoloV11                                                                                                          |
| **Integrantes** | Bruno Masoller (brunomaso1@gmail.com)                                                                                                  |

</div>

## Entrenamiento del modelo

Creamos el modelo de detección de objetos YOLOv11:

<small><em>https://docs.ultralytics.com/modes/train/#train-settings</em></small>

In [5]:
# model_name = "yolo11n"
# # model_name = "yolo11x"
# model_path = model_name + ".pt"
# project_name = f"deteccion_palmeras_{model_name}_{str(uuid.uuid4()).split('-')[-1]}"

# model = YOLO(model_path)
# LOGGER.info(f"Ejecutando el entrenamiento del modelo {model_name} en el proyecto {project_name}")

In [6]:
project_name = "deteccion_palmeras_yolo11n_79b7c0f7f315"

In [7]:
# results = model.train(
#     data=str(DOWNLOAD_YOLO_DATASET_FOLDER / "dataset.yaml"),
#     epochs=100,
#     batch=64,
#     device=0,
#     imgsz=640,
#     project=project_name,
#     seed=CONFIG["seed"],
#     exist_ok=True,
#     save_period=1,
#     patience=10,
#     # cache=True,
#     # plots=True,
# )

# results = model.train(data="coco128.yaml", epochs=100, imgsz=640, project=project_name, exist_ok=True)

## Validación del modelo

Cargamos el mejor modelo:

In [8]:
# project_name = "deteccion_palmeras_yolo11n_5fb7f6c117b1"
print(f"Nombre del proyecto: {project_name}")
best_model_path = Path(project_name) / "train" / "weights" / "best.pt"

if not best_model_path.exists():
    LOGGER.error(f"El modelo {best_model_path} no existe")
    sys.exit(1)
best_model = YOLO(best_model_path)

Nombre del proyecto: deteccion_palmeras_yolo11n_79b7c0f7f315


Limpiamos la GPU:

In [9]:
GPUUtils.clean_gpu_usage()

Initial GPU Usage
| ID | GPU | MEM |
------------------
|  0 | 11% |  8% |
GPU Usage after emptying the cache
| ID | GPU | MEM |
------------------
|  0 | 10% |  8% |


Validamos el modelo:

In [10]:
metrics = best_model.val(project=project_name)

Ultralytics 8.3.140  Python-3.13.3 torch-2.7.0+cu126 CUDA:0 (NVIDIA GeForce RTX 4080 SUPER, 16376MiB)
YOLO11n summary (fused): 100 layers, 2,616,248 parameters, 0 gradients, 6.5 GFLOPs
val: Fast image access  (ping: 0.00.0 ms, read: 868.4242.5 MB/s, size: 41.7 KB)


val: Scanning E:\Documentos\Git Repositories\datasets\coco128\labels\train2017.cache... 126 images, 2 backgrounds, 0 corrupt: 100%|██████████| 128/128 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 8/8 [00:01<00:00,  6.29it/s]


                   all        128        929      0.917      0.811      0.894      0.748
                person         61        254      0.981      0.713      0.877      0.709
               bicycle          3          6      0.565        0.5      0.599      0.402
                   car         12         46      0.946      0.382       0.56      0.319
            motorcycle          4          5      0.936          1      0.995      0.916
              airplane          5          6      0.943          1      0.995      0.961
                   bus          5          7          1      0.787      0.995      0.897
                 train          3          3      0.917          1      0.995       0.93
                 truck          5         12       0.93        0.5      0.767      0.518
                  boat          2          6      0.947      0.667      0.811      0.636
         traffic light          4         14          1      0.377      0.489      0.324
             stop sig

Imprimimos el reporte de validación:

In [11]:
metrics.summary()

[{'class_name': 'person',
  'box-p': np.float64(0.980531943575975),
  'box-r': np.float64(0.7125984251968503),
  'box-f1': np.float64(0.8253652899202091),
  'box-map': np.float64(0.7480672061581388),
  'box-map50': np.float64(0.8935084476532619),
  'box-map75': np.float64(0.8193343490072319)},
 {'class_name': 'bicycle',
  'box-p': np.float64(0.5652587667426311),
  'box-r': np.float64(0.5),
  'box-f1': np.float64(0.5306304762654902),
  'box-map': np.float64(0.7480672061581388),
  'box-map50': np.float64(0.8935084476532619),
  'box-map75': np.float64(0.8193343490072319)},
 {'class_name': 'car',
  'box-p': np.float64(0.946152897435272),
  'box-r': np.float64(0.3822671939752834),
  'box-f1': np.float64(0.5445313805666997),
  'box-map': np.float64(0.7480672061581388),
  'box-map50': np.float64(0.8935084476532619),
  'box-map75': np.float64(0.8193343490072319)},
 {'class_name': 'motorcycle',
  'box-p': np.float64(0.9359790265688755),
  'box-r': np.float64(1.0),
  'box-f1': np.float64(0.96693

## Probamos la predicción

Cargamos el modelo:

In [12]:
print(f"Nombre del proyecto: {project_name}")
best_model_path = Path(project_name) / "train" / "weights" / "best.pt"

if not best_model_path.exists():
    LOGGER.error(f"El modelo {best_model_path} no existe")
    sys.exit(1)
best_model = YOLO(best_model_path)

Nombre del proyecto: deteccion_palmeras_yolo11n_79b7c0f7f315


### Parche

Descargamos el parche:

In [13]:
# patch_name = "000000000192"
patch_name = "000000000074"
patch_file_path = DOWNLOAD_PREDICTION_FOLDER / f"{patch_name}.jpg"

Mostramos el parche:

In [14]:
# image = cv.imread(str(patch_file_path), cv.IMREAD_COLOR)
# cv.imshow("Imagen", image)
# cv.waitKey(0)
# cv.destroyAllWindows()

Realizamos la predicción:

In [15]:
results = best_model.predict(patch_file_path, project=project_name, save=True, save_txt=True, name=patch_name)


image 1/1 e:\Documentos\Git Repositories\uba-ceia-proy-final\ceia-proyecto-final\modulo-IA\test\yolo11\downloads\predictions\000000000074.jpg: 448x640 7 persons, 1 bicycle, 1 dog, 39.3ms
Speed: 1.0ms preprocess, 39.3ms inference, 1.4ms postprocess per image at shape (1, 3, 448, 640)
Results saved to deteccion_palmeras_yolo11n_79b7c0f7f315\0000000000748
1 label saved to deteccion_palmeras_yolo11n_79b7c0f7f315\0000000000748\labels


Obtenemos los resultados:

In [16]:
for result in results:
    xywh = result.boxes.xywh  # center-x, center-y, width, height
    xywhn = result.boxes.xywhn  # normalized
    xyxy = result.boxes.xyxy  # top-left-x, top-left-y, bottom-right-x, bottom-right-y
    xyxyn = result.boxes.xyxyn  # normalized
    names = [result.names[cls.item()] for cls in result.boxes.cls.int()]  # class name of each box
    confs = result.boxes.conf  # confidence score of each box

    print(f"Resultados para el parche: {patch_name}")
    print(f"Cantidad de detecciones: {len(names)}")
    if len(names) == 0:
        print("No se detectaron objetos.")
    else:
        for i, (name, conf, box) in enumerate(zip(names, confs, xyxy)):
            print(f"Detección {i+1}: Clase={name}, Confianza={conf.item():.2f}, Caja={box.tolist()}")

Resultados para el parche: 000000000074
Cantidad de detecciones: 9
Detección 1: Clase=bicycle, Confianza=0.95, Caja=[0.0247344970703125, 5.4251708984375, 162.5913848876953, 319.46917724609375]
Detección 2: Clase=dog, Confianza=0.93, Caja=[62.925567626953125, 276.4197692871094, 358.3177185058594, 379.1241760253906]
Detección 3: Clase=person, Confianza=0.65, Caja=[461.56817626953125, 104.64024353027344, 493.97003173828125, 148.62413024902344]
Detección 4: Clase=person, Confianza=0.61, Caja=[295.22723388671875, 95.46449279785156, 314.08782958984375, 152.9888153076172]
Detección 5: Clase=person, Confianza=0.60, Caja=[325.35723876953125, 95.34477233886719, 341.44659423828125, 147.47666931152344]
Detección 6: Clase=person, Confianza=0.58, Caja=[276.23223876953125, 98.58846282958984, 292.928466796875, 152.75057983398438]
Detección 7: Clase=person, Confianza=0.47, Caja=[354.0404357910156, 96.23331451416016, 372.2051696777344, 148.19137573242188]
Detección 8: Clase=person, Confianza=0.40, Caja=

Convertimos a anotaciones COCO:

In [17]:
coco_annotations = YoloDatasetUtils.create_coco_annotations_from_yolo_results(results, patch_name)

Creamos la capa KML:

In [ ]:
# TODO

### Imagen

In [ ]:
image_name = "8deOctubreyCentenario-EspLibreLarranaga_20190828_dji_pc_5cm_patch_0"
Path(DOWNLOAD_PREDICTION_FOLDER).mkdir(parents=True, exist_ok=True)
image_name_path = DOWNLOAD_PREDICTION_FOLDER / f"{image_name}.jpg"
ProcesadorS3 = ProcesadorS3()
ProcesadorS3.download_patch_from_minio(image_name, image_name_path)
LOGGER.debug(f"Se descargó el parche {image_name_path} desde MinIO")